<a href="https://colab.research.google.com/github/Ads369/Ads_2s/blob/main/7_3_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Библиотеки Python для Data Science](https://colab.research.google.com/drive/1VKRkuVNaSRPy6uNRhWLIlfaTnLPuyWcC)
2. [Отзывы к фильмам (исследуем датасет IMDB)](https://colab.research.google.com/drive/1jL4XbkAMuMNVPXhfsE8Z2Z4h-sGtyAq1)
3. Домашняя работа

Используя базу данных фильмов из практической части урока, проверте следующие гипотезы:

1. Большинство фильмов выпускаются по пятницам
2. Известные актеры снимаются в самых кассовых фильмах
3. Известные актеры снимаются в самыx дорогих фильмах

Построить графики по гипотезам. Сколько актеров из вашего результата вам знакомы?

In [ ]:
from requests import get

response = get("https://storage.yandexcloud.net/academy.ai/the_movies_dataset.zip")
with open("the_movies_dataset.zip", "wb") as f:
    f.write(response.content)

# Разархивация датасета в директорию 'the_movies_dataset'
!unzip -qo "the_movies_dataset.zip" -d ./the_movies_dataset

In [ ]:
import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

DIR_PATH = "./the_movies_dataset"
WEEK_DAYS_ORDER = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

warnings.filterwarnings("ignore")


def load_data(file_name: str) -> pd.DataFrame:
    return pd.read_csv(f"{DIR_PATH}/{file_name}")


def count_movies_by_weekday(df: pd.DataFrame) -> pd.DataFrame:
    sns.countplot(x="release_weekday", data=df, order=WEEK_DAYS_ORDER)
    plt.show()


def _data_to_weekday(df: pd.DataFrame) -> pd.DataFrame:
    # Не использую lambda, т.к. в данном случае c ней программа работает дольше
    df = df.drop(df[df["release_date"].isnull()].index)
    df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
    df["release_weekday"] = df["release_date"].dt.day_name()
    return df


def _prepare_numeric_columns(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors="coerce")
        df[column] = df[column].fillna(0)
        df = df.drop(df[df[column].isnull()].index)
    return df


def get_actors(cast: str) -> list:
    actors = []
    for i in ast.literal_eval(cast):
        actors.append(i["name"])
    return actors


In [ ]:

move_metadata = load_data("movies_metadata.csv")
move_metadata = move_metadata[["id", "release_date", "budget", "revenue"]]
move_metadata = _data_to_weekday(move_metadata)
move_metadata = _prepare_numeric_columns(move_metadata, ["id", "budget", "revenue"])

credits_metadata = load_data("credits.csv")
credits_metadata = _prepare_numeric_columns(credits_metadata, ["id"])

merged_metadata = move_metadata.merge(credits_metadata, on="id")
merged_metadata["actors"] = merged_metadata["cast"].apply(get_actors)
merged_metadata = merged_metadata[["actors", "budget", "revenue"]]

In [ ]:
## №1
# count_movies_by_weekday(move_metadata)
sns.countplot(x="release_weekday", data=move_metadata, order=WEEK_DAYS_ORDER)
plt.show()


In [ ]:
## №2
plt.title("Актеры с лучшими показателями по кассовым сборам")
merged_metadata.explode("actors").groupby("actors")["revenue"].sum().sort_values(
    ascending=False
).head(10).plot(kind="bar")
plt.show()

In [ ]:
## №3
plt.title("Актеры в самых бюджетных фильмах")
merged_metadata.explode("actors").groupby("actors")["budget"].sum().sort_values(
    ascending=False
).head(10).plot(kind="bar")
plt.show()